### Importer les librairies

In [5]:
import torch
from datasets import load_dataset
from transformers import pipeline, BertTokenizer, BertModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import sent_tokenize

2023-11-15 22:09:42.823502: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-15 22:09:42.997537: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 22:09:42.997569: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 22:09:42.998774: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 22:09:43.083963: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

### Charger le dataset

In [10]:
dataset = load_dataset('cnn_dailymail', '3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /home/olivier/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

### Prendre un sous-ensemble de données pour un traitement plus rapide

In [11]:
subset_size = 100
articles = dataset['test'][:subset_size]['article']
summaries = dataset['test'][:subset_size]['highlights']

### Tokenization && embedding

In [12]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
embeddings = []

for article in articles:
    sentences = sent_tokenize(article)
    tokenized_sentences = [tokenizer(sentence, return_tensors='pt') for sentence in sentences]
    with torch.no_grad():
        sentence_embeddings = [model(**sentence)['last_hidden_state'][:, 0, :].numpy() for sentence in tokenized_sentences]
        document_embedding = sum(sentence_embeddings) / len(sentence_embeddings)
        embeddings.append(document_embedding)

embeddings = torch.tensor(embeddings)

/tmp/ipykernel_5328/1226251185.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  embeddings = torch.tensor(embeddings)


In [14]:
# Afficher la forme de la matrice d'incorporation
print("Shape of embeddings:", embeddings.shape)

Shape of embeddings: torch.Size([100, 1, 768])


### Calcul de similarité et summarization

In [15]:
# Calculer la similarité cosinus
cosine_similarities = cosine_similarity(embeddings.squeeze(1).numpy())

# Build the summary by selecting top-ranked sentences
summary_length = 3  # Number of sentences in the summary
selected_sentences = []

for i in range(summary_length):
    sentence_scores = cosine_similarities[i]
    selected_sentence_index = sentence_scores.argmax()
    selected_sentences.append(articles[i].split('.')[selected_sentence_index])

# Combine selected sentences into a summary
generated_summary = ' '.join(selected_sentences)

# Afficher le résumé généré
print("\nGenerated Summary:")
print(generated_summary)


Generated Summary:
(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories  A stray pooch in Washington State has used up at least three of her own after being hit by a car, apparently whacked on the head with a hammer in a misguided mercy killing and then buried in a field -- only to survive  He has been U
